## A helpful AI Assistant for RGV Fishing Area and Waterway Cleanups

* **Created by:** Paul Hitchcox
* **For:** CSCI 3351
* **At:** University of Texas Rio-Grande Valley

## Setup

In [34]:
%pip install -U --quiet pydantic openai gradio geocoder

Note: you may need to restart the kernel to use updated packages.


## Database

In [4]:
def create_database():
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('cleanup_schedule.db')
    cursor = conn.cursor()

    # Create tables
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS events (
        event_id INTEGER PRIMARY KEY AUTOINCREMENT,
        event_name TEXT NOT NULL,
        location TEXT NOT NULL,
        event_date TEXT NOT NULL
    );
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS volunteers (
        volunteer_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        phone TEXT NOT NULL
    );
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS registered_volunteers (
        volunteer_id INTEGER,
        event_id INTEGER,
        FOREIGN KEY(volunteer_id) REFERENCES volunteers(volunteer_id),
        FOREIGN KEY(event_id) REFERENCES events(event_id)
    );
    ''')

    # Commit changes
    conn.commit()

    # Generate and insert cleanup events
    generate_cleanup_events(cursor)

    # Commit changes and close the connection
    conn.commit()
    conn.close()

    print("Database and tables created successfully, and initial data inserted.")

def generate_cleanup_events(cursor):
    # Define the events you want to create
    events = [
        {"event_name": "Puente de los Lobos and Gayman's Bridge Cleanup", "location": "Highway 48, Brownsville, Texas", "event_date": "2024-05-18"},
        {"event_name": "Boca Chica Beach Cleanup", "location": "Highway 4, Brownsville, Texas", "event_date": "2024-10-25"},
        {"event_name": "Jaime J. Zapata Boat Ramp", "location": "Highway 48, Brownsville, Texas", "event_date": "2024-9-8"}
    ]

    for event in events:
        cursor.execute('INSERT INTO events (event_name, location, event_date) VALUES (?, ?, ?)',
                       (event["event_name"], event["location"], event["event_date"]))

create_database()

Database and tables created successfully, and initial data inserted.


In [3]:
from datetime import datetime
from datetime import timedelta
import sqlite3
import re
import geocoder

def connect_db():
    return sqlite3.connect('cleanup_schedule.db')

def validate_phone_number(phone):
    pattern = re.compile(r"^\(\d{3}\) \d{3}-\d{4}$")
    return pattern.match(phone) is not None

def validate_date(date_text):
    try:
        datetime.strptime(date_text, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def validate_address(address):
    # Use the geocoder to get the location data
    g = geocoder.osm(address)
    if g.ok:
        # Check if the address is in Edinburg, TX
        if 'Edinburg' in g.city and 'Texas' in g.state:
            return True, g.latlng
        else:
            return False, None
    return False, None

def is_slot_available(slot_id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('SELECT event_id FROM registered_volunteers WHERE event_id = ?', (slot_id,))
    result = cursor.fetchone()
    conn.close()
    return result is None

def register_volunteer(name: str, email: str, phone: str, event_id: int):
    "Attempts to register a volunteer for a cleanup event. All fields are required."

    if not validate_phone_number(phone):
        return "Invalid phone number. Please provide a valid phone number in the format (XXX) XXX-XXXX."

    valid_address, coordinates = validate_address(email)
    if not valid_address:
        return "Invalid email address."

    conn = connect_db()
    cursor = conn.cursor()

    try:
        # Insert volunteer if not exists
        cursor.execute('INSERT OR IGNORE INTO volunteers (name, email, phone) VALUES (?, ?, ?)', (name, email, phone))
        volunteer_id = cursor.lastrowid

        if volunteer_id == 0:
            cursor.execute('SELECT volunteer_id FROM volunteers WHERE email = ?', (email,))
            volunteer_id = cursor.fetchone()[0]

        # Register the volunteer for the event
        cursor.execute('INSERT INTO registered_volunteers (volunteer_id, event_id) VALUES (?, ?)', (volunteer_id, event_id))

        conn.commit()
    except sqlite3.Error as e:
        conn.rollback()
        conn.close()
        return f"An error occurred: {e}"
    finally:
        conn.close()

    return "Successfully registered as a volunteer for the cleanup event."

def get_available_events():
    "Returns all available cleanup events"
    conn = connect_db()
    cursor = conn.cursor()

    cursor.execute('SELECT event_id, event_name, location, event_date FROM events')
    events = cursor.fetchall()

    conn.close()

    return [{"event_id": event_id, "event_name": event_name, "location": location, "event_date": event_date} for event_id, event_name, location, event_date in events]

def get_registered_volunteers():
    "Returns all registered volunteers for cleanup events"
    conn = connect_db()
    cursor = conn.cursor()

    cursor.execute('''SELECT rv.volunteer_id, v.name, v.email, v.phone, e.event_name, e.location, e.event_date
                      FROM registered_volunteers rv
                      JOIN volunteers v ON rv.volunteer_id = v.volunteer_id
                      JOIN events e ON rv.event_id = e.event_id
                      ORDER BY e.event_date''')

    volunteers = cursor.fetchall()
    conn.close()

    columns = ["volunteer_id", "name", "email", "phone", "event_name", "location", "event_date"]
    volunteers_data = [dict(zip(columns, volunteer)) for volunteer in volunteers]

    return volunteers_data

# Example usage
print(get_registered_volunteers())

[]


## Define Tools

In [5]:
import requests

def geocode(city_name:str):
    "Geocodes a city name into latitude and longitude data"
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=10&language=en&format=json'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
print(geocode(city_name="Edinburg"))

{'results': [{'id': 4688275, 'name': 'Edinburg', 'latitude': 26.30174, 'longitude': -98.16334, 'elevation': 30.0, 'feature_code': 'PPLA2', 'country_code': 'US', 'admin1_id': 4736286, 'admin2_id': 4697444, 'timezone': 'America/Chicago', 'population': 84497, 'postcodes': ['78539', '78540'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Texas', 'admin2': 'Hidalgo'}, {'id': 4257043, 'name': 'Edinburgh', 'latitude': 39.35422, 'longitude': -85.96666, 'elevation': 205.0, 'feature_code': 'PPL', 'country_code': 'US', 'admin1_id': 4921868, 'admin2_id': 4259727, 'admin3_id': 4254728, 'timezone': 'America/Indiana/Indianapolis', 'population': 4546, 'postcodes': ['46124'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Indiana', 'admin2': 'Johnson', 'admin3': 'Blue River Township'}, {'id': 4757240, 'name': 'Edinburg', 'latitude': 38.82095, 'longitude': -78.56585, 'elevation': 246.0, 'feature_code': 'PPL', 'country_code': 'US', 'admin1_id': 6254928, 'admin2_id': 4785252

In [6]:
def weather(latitude:float, longitude:float):
    "Returns the weather conditions for a given latitude and longitude"
    url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,is_day,precipitation,rain,showers,snowfall&timezone=America%2FChicago'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
print(weather(latitude=26.30174, longitude=-98.16334))

{'latitude': 26.309208, 'longitude': -98.153625, 'generationtime_ms': 0.030040740966796875, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 30.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'is_day': '', 'precipitation': 'mm', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'current': {'time': '2024-05-19T20:00', 'interval': 900, 'temperature_2m': 31.4, 'is_day': 1, 'precipitation': 0.0, 'rain': 0.0, 'showers': 0.0, 'snowfall': 0.0}}


In [7]:
from datetime import datetime
from datetime import timedelta

# gets todays date
def get_todays_date():
    "Gets today's date"
    return datetime.now().strftime('%B %d, %Y')

# Example:
print(get_todays_date())

May 19, 2024


In [8]:
# gets tomorrows date
def get_tomorrows_date():
    "Gets tomorrow's date"
    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    return tomorrow.strftime('%B %d, %Y')

print(get_tomorrows_date())

May 20, 2024


In [9]:
# gets the current time
def get_current_time():
    "Gets the current time"
    return datetime.now().strftime("%m/%d/%Y %I:%M %p")

# Example:
print(get_current_time())

05/19/2024 08:02 PM


## Format for OpenAI

In [10]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

def schema(f):
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
          for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).model_json_schema()
    function_schema = dict(name=f.__name__, description=f.__doc__, parameters=s)
    return {
        "type": "function",
        "function": function_schema
    }

def call_func(name, arguments):
    f = globals()[name]
    return f(**json.loads(arguments))

In [11]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)  # take environment variables from .env.

from openai import OpenAI

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY")
)

def chat_completion(
    message,
    model="gpt-4",
    prompt="You are a helpful assistant.",
    temperature=0,
    messages=[],
    tools=[]
):
    # Add the prompt to the messages list
    if prompt is not None:
        messages = [{"role": "system", "content": prompt}] + messages

    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]

    # NEW
    if tools:
        tool_schemas = [schema(f) for f in tools]
    else:
        tool_schemas = None
        
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    print(f"messages: {messages}")
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        tools=tool_schemas
    )
    
    # Updated
    assistant_message = completion.choices[0].message
        
    return assistant_message

In [12]:
chat_completion("Add 2+3", model="gpt-4", tools=[sum])

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class 'inspect._empty'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.7/u/schema-for-unknown-type

In [15]:
# Define a function to handle the chat interaction with the AI model
def chat(message, chatbot_messages, history_state):
    prompt = """
    You are a helpful AI assistant named Sparky created by RGV Fishing Area and Waterway Cleanups, a nonprofit organiztion in South Texas.

    Today's date: {todays_date}
    Current time: {current_time}
    
    ## Task
    
    Your job is to provide users and volunteers for waterway cleanup events with helpful and accurate information.
    
    Additionally, you will inform them of the board of directors, upcoming events, and sign people up for cleanups.
    
    As an AI assistant, you will make sure that people are not allowed to sign up for a past cleanup event.
    
    ## General Info
    
    RGV Fishing Area and Waterway cleanups was founded in 2018 when Richard Hitchcox decided to host his event at Gayman's bridge on Highway 48
    in Brownsville. He has since hosted dozens of cleanups, established a board of directors, established the group as a Texas Domestic Nonprofit
    organization, and has attracted attention to his cause through social media posts and local television interviews. 

    The link to group's Facebook page is: https://www.facebook.com/RGVFAWC
    
   
    
    ## Board of Directors

    Richard Hitchcox - Managing Director and Founder - Richard is a retired US Army veteran and former forensic scientist for Texas DPS. 
    He founded the cleanup group in 2018 after relaizing the need for clenaing up the environment while fihsing with his son Paul. He 
    has recently suffered a heart attack and underwent open heart surgery, so he is doing primarily administrative work for the group right now.

    Paul - Hitchcox - Director - Paul is a 20 year old college student studying computer science at UTRGV in Edinburg, Texas. He is the son of 
    Richard Hitchcox, and he is the former president of the RGV Fishing Area and Waterway Cleanups Youth Advisory Council. When he turned 18, he 
    was made a Director. He helps manage registration, takes photos at events, and helps load and unload trucks as they fill up the dumpsters. 
    He also is the creator of Sparky.

    Carlos Salinas - Director - Carlos Salinas is a former mechanical engineer and now local businessowner of Brownsville Bait and Tackle. He 
    has cooked hot dogs for volunteers at past events and has got the support of the local fishing industry through connections to boat captains
    and lure makers in the RGV. He has also gone live on Facebook numerous times to promote upcoming events and has made promotional videos for the group.

    Ramon de la Fuente - Director - Ramon is a mechanic and co-owner of De la Fuente Mechanic Shop in Brownsville, Texas. Ramon always brings 
    his large family out and fills up dumpsters full of trash his family and church community that he invites. He has cooked numerous chicken
    plates and sides for the volunteers at some events, and he has helped get people unstuck from sand at Boca Chica Beach numerous times.
    Ramon is very hard working.

    Henry Rodriguez - Director - Henry is a local fishing captain, tour guide in Nuevo Progreso Tamaulipas, and tour guide for local vineyards in South
    Texas. Henry has organized various events, created various decals and logos for the group, promoted the group on Facebook, and has brought 
    in a lot of clients from his jobs to come and support the cause. He has also led boat cleanups in order to cleanup that are inaccessible
    by land by coordinating with local Texas paddling groups and other captains.

    Noe Gonzalez - Director - Noe is a local fishing captain and the founder of the Bubba's South Texas Sportsman's group. Noe underwent a liver
    transplant in 2017 after nearly dying due to extreme liver disease. With his new lease on life, Noe is giving everything he can to the 
    community, and he is very passionate about keeping Texas waterways clean. He uses his connections as a boat captain to attract volunteers
    and raise support for the group.


        
    ## General Cleanup Information

    At cleanups, volunteers can expect to sign a waiver form, be entered into a free raffle just for volunteering, enjoy a nice day of cleaning
    up trash, be provided with sunscreen, insect repellant, reacher grabbers, trash bags, work gloves, nitrile gloves, snacks, waters, gatorade, and first aid,
    and a warm and inviting atmosphere to return for further events. Most cleanup events go from 8 AM - 11 AM at various cleanup locations across 
    the valley. We often partner with Marine Military Academy, Girl Scouts of South Texas, Harmony School of Innovation, and various cleanup groups,
    rotary clubs, and local government entities to accomplish our goals.
    

    
    ## RGV Fishing Area and Waterway Cleanups Opportunities

    Withing RGV Fishing Area and Waterway Cleanups, we have a youth advisory council that assists the board of directors in coordinating
    cleanup events and getting youth in the RGV excited about cleanups. We also award and present the President's Volunteer Service Award to 
    individuals that meet the requirements based on volunteer hours. We also present certificates of appreciation to our local sponsors and 
    partners. In addition, we hold the annual South Texas Angler's Tournament (STAT) in order to raise funds for future cleanups and provide 
    families with a fun and safe fishing experience.



    ## Signing up Users for Cleanups

    You are being given the ability to access a database with tables for events and for registered volunteers. You may access the one with events
    if asked about registering for future events. You can also get user input to register for them in the registered volunteers table. You can 
    show them past events by accessing the table. Make sure to ask for the email, phone number, and other fields before allowing them to register
    
    
    
    """.format(
        todays_date=get_todays_date(),
        current_time=get_current_time()
    )
    
    # Initialize chatbot_messages and history_state if they are not provided
    allowed_tools = [sum, weather, get_tomorrows_date, get_available_events_df, get_registered_volunteers_df]
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    
    # Try to get the AI's reply using the chat_completion
    try:
        assistant_message = chat_completion(message, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    # UPDATED
    while(assistant_message.tool_calls):
        for tool_call in assistant_message.tool_calls:
            tool_call_json = json.dumps({ "name": tool_call.function.name, "arguments": tool_call.function.arguments})
            history_state.append({"role": assistant_message.role, "content": tool_call_json})
            
        for tool_call in assistant_message.tool_calls:
            results = call_func(tool_call.function.name, tool_call.function.arguments)
            results_str = str(results) # convert to string
            history_state.append({"role": "function", "tool_call_id": tool_call.id, "name": tool_call.function.name, "content": results_str})

        # Try to get the AI's reply using the get_ai_reply function
        try:
            assistant_message = chat_completion(None, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
        except Exception as e:
            # If an error occurs, raise a Gradio error
            raise gr.Error(e)
        
    # Append the user's message and the AI's reply to the chatbot_messages list
    if(assistant_message.content):
        chatbot_messages.append((message, assistant_message.content.strip()))
    else:
        chatbot_messages.append((message, None))
    
    # Append the user's message and the AI's reply to the history_state list
    history_state.append({"role": "user", "content": message})
    history_state.append({"role": "assistant", "content": assistant_message.content})
    
    # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    return None, chatbot_messages, history_state

In [16]:
import gradio as gr
import pandas as pd


def get_available_events_df():
    # Get available events data as a list of dictionaries
    events_data = get_available_events()

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(events_data)

    return df

def get_registered_volunteers_df():
    # Get registered volunteers data as a list of dictionaries
    volunteers_data = get_registered_volunteers()

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(volunteers_data)

    return df

# Define a function to return a chatbot app using Gradio
def get_chatbot_app(share=False):
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        with gr.Tab("Conversation"):
            with gr.Row():
                with gr.Column():
                    # Create a chatbot interface for the conversation
                    chatbot = gr.Chatbot(label="Conversation")
                    # Create a textbox for the user's message
                    message = gr.Textbox(label="Message")
                    # Create a state object to store the conversation history
                    history_state = gr.State()
                    # Create a state object to store the available tools
                    tools = gr.State()
                    # Create a button to send the user's message
                    btn = gr.Button(value="Send")
                btn.click(chat, inputs=[message , chatbot, history_state,], outputs=[message, chatbot, history_state])
        with gr.Tab("(Admin) Registered Volunteers"):
            refresh = gr.Button(value="Refresh")
            volunteers = gr.Dataframe(value=get_registered_volunteers_df())
            refresh.click(get_registered_volunteers_df, outputs=volunteers)
        with gr.Tab("(Admin) Update Volunteer"):
            volunteer_events = gr.Dataframe(value=get_available_events_df())
        # Return the app
        return app

# Call the launch_chatbot function to start the chatbot interface using Gradio
# Set the share parameter to False, meaning the interface will not be publicly accessible
get_chatbot_app().launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/var/folders/08/zr90r6nj03n7f9gsvb46cbt00000gn/T/ipykernel_92857/225651952.py", line 99, in chat
    assistant_message = chat_completion(message, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/08/zr90r6nj03n7f9gsvb46cbt00000gn/T/ipykernel_92857/3470031669.py", line 31, in chat_completion
    tool_schemas = [schema(f) for f in tools]
                    ^^^^^^^^^
  File "/var/folders/08/zr90r6nj03n7f9gsvb46cbt00000gn/T/ipykernel_92857/3915492106.py", line 8, in schema
    s = create_model(f'Input for `{f.__name__}`', **kw).model_json_schema()
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/jupyterlab/4.1.2/libexec/lib/python3.12/site-packages/pydantic/main.py", line 1550, in create_model
    return meta(
           ^^^^^
  File "/opt/home